<a href="https://colab.research.google.com/github/AbhiJeet70/Attack_GraphNN/blob/main/Poison_Cora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/DSE-MSU/DeepRobust.git
%cd DeepRobust
!pip install gensim==3.8.3
!python setup.py install

import os
import collections
!pip install torch
import torch
!pip install torch_geometric
import torch_geometric
from torch import Tensor
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import statistics


# DATA SPLIT
import torch_geometric.transforms as T
from deeprobust.graph.data import Dataset, PrePtbDataset, Dpr2Pyg

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(Net, self).__init__()
    self.conv1 = GCNConv(input_dim, hidden_dim)
    self.conv2 = GCNConv(hidden_dim, output_dim)

  def forward(self, data):
    x, edge_index = data.x, data.edge_index

    x = self.conv1(x, edge_index)
    x = F.relu(x)
    x = F.dropout(x, training = self.training)
    x = self.conv2(x, edge_index)

    return F.log_softmax(x, dim = 1)

# You can either use setting='prognn' or seed=15 to get the prognn splits
#data = Dataset(root='/tmp/', name='cora', setting='prognn')
data = Dataset(root='/tmp/', name='cora') # load clean graph
pyg_data = Dpr2Pyg(data) # convert dpr to pyg
print('DATA')
print(pyg_data)


#average = []
#for i in range(1, 51):
model = Net(pyg_data.num_node_features, 512, pyg_data.num_classes).to(device)
data = pyg_data[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas = (0.9, 0.999), eps=1e-08, weight_decay = 0)

model.train()
for epoch in range(1,501):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    print(f'Epoch {epoch:03d}, Loss {loss:.4f}')
    loss.backward()
    optimizer.step()

model.eval()
_, pred = model(data).max(dim=1)
correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print(f'Accuracy: {acc:.4f}')
#average.append(acc)
#print(average)
#print(f'Average Accuracy: {statistics.mean(average):.4f}')

Cloning into 'DeepRobust'...
remote: Enumerating objects: 5835, done.
remote: Counting objects: 100% (1547/1547), done.
remote: Compressing objects: 100% (645/645), done.
remote: Total 5835 (delta 968), reused 1461 (delta 894), pack-reused 4288
Receiving objects: 100% (5835/5835), 12.09 MiB | 13.48 MiB/s, done.
Resolving deltas: 100% (3831/3831), done.
/content/DeepRobust
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 33.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for gensim
  Running setup.py clean for gensim
Failed to build gensim
ERROR: Could not build wheels for gensim, which is required to install pyproject.toml-based projects
running install
/usr/local/lib/python3.10/dist-packages/setuptools/

Processing...
/content/DeepRobust/deeprobust/graph/data/pyg_dataset.py:48: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  edge_index = torch.LongTensor(dpr_data.adj.nonzero())
Done!


DATA
Dpr2Pyg()


/usr/local/lib/python3.10/dist-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Epoch 001, Loss 1.0968
Epoch 002, Loss 1.0162
Epoch 003, Loss 0.9453
Epoch 004, Loss 0.8739
Epoch 005, Loss 0.7932
Epoch 006, Loss 0.7096
Epoch 007, Loss 0.6422
Epoch 008, Loss 0.5832
Epoch 009, Loss 0.5286
Epoch 010, Loss 0.4809
Epoch 011, Loss 0.4409
Epoch 012, Loss 0.4176
Epoch 013, Loss 0.3991
Epoch 014, Loss 0.3884
Epoch 015, Loss 0.3759
Epoch 016, Loss 0.3711
Epoch 017, Loss 0.3624
Epoch 018, Loss 0.3615
Epoch 019, Loss 0.3537
Epoch 020, Loss 0.3458
Epoch 021, Loss 0.3339
Epoch 022, Loss 0.3337
Epoch 023, Loss 0.3325
Epoch 024, Loss 0.3178
Epoch 025, Loss 0.3117
Epoch 026, Loss 0.3002
Epoch 027, Loss 0.2994
Epoch 028, Loss 0.2855
Epoch 029, Loss 0.2816
Epoch 030, Loss 0.2767
Epoch 031, Loss 0.2730
Epoch 032, Loss 0.2630
Epoch 033, Loss 0.2578
Epoch 034, Loss 0.2524
Epoch 035, Loss 0.2477
Epoch 036, Loss 0.2404
Epoch 037, Loss 0.2371
Epoch 038, Loss 0.2339
Epoch 039, Loss 0.2314
Epoch 040, Loss 0.2257
Epoch 041, Loss 0.2214
Epoch 042, Loss 0.2163
Epoch 043, Loss 0.2153
Epoch 044, 

In [ ]:
# Load meta attacked data
perturbed_data = PrePtbDataset(root='/tmp/', name='cora', attack_method='meta', ptb_rate=0.05)
perturbed_adj = perturbed_data.adj
pyg_data.update_edge_index(perturbed_adj) # inplace operation

print('PERTURBATED DATA')
print(pyg_data)
print(len(pyg_data))
print(pyg_data[0])

model = Net(pyg_data.num_node_features, 512, pyg_data.num_classes).to(device)
data = pyg_data[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas = (0.9, 0.999), eps=1e-08, weight_decay = 0)

model.train()
average = []
for i in range(1, 51):
  for epoch in range(1,501):
      optimizer.zero_grad()
      out = model(data)
      loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
      print(f'Epoch {epoch:03d}, Loss {loss:.4f}')
      loss.backward()
      optimizer.step()

  model.eval()

  _, pred = model(data).max(dim=1)
  correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
  acc = correct / data.test_mask.sum().item()
  print(f'Accuracy: {acc:.4f}')
  average.append(acc)
  print(average)
print(f'Average Accuracy: {statistics.mean(average):.4f}')

Loading pubmed dataset perturbed by 0.05 meta...
PERTURBATED DATA
Dpr2Pyg()
1
Data(x=[19717, 500], edge_index=[2, 93080], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])


/content/DeepRobust/deeprobust/graph/data/attacked_data.py:135: UserWarning: The pre-attacked graph is perturbed under the data splits provided by ProGNN. So if you are going to verify the attacking performance, you should use the same data splits  (setting='prognn').
  warnings.warn("The pre-attacked graph is perturbed under the data splits provided by ProGNN. So if you are going to verify the attacking performance, you should use the same data splits  (setting='prognn').")


Streaming output truncated to the last 5000 lines.
Epoch 415, Loss 0.0000
Epoch 416, Loss 0.0000
Epoch 417, Loss 0.0000
Epoch 418, Loss 0.0000
Epoch 419, Loss 0.0000
Epoch 420, Loss 0.0000
Epoch 421, Loss 0.0000
Epoch 422, Loss 0.0000
Epoch 423, Loss 0.0000
Epoch 424, Loss 0.0000
Epoch 425, Loss 0.0000
Epoch 426, Loss 0.0000
Epoch 427, Loss 0.0000
Epoch 428, Loss 0.0000
Epoch 429, Loss 0.0000
Epoch 430, Loss 0.0000
Epoch 431, Loss 0.0000
Epoch 432, Loss 0.0000
Epoch 433, Loss 0.0000
Epoch 434, Loss 0.0000
Epoch 435, Loss 0.0000
Epoch 436, Loss 0.0000
Epoch 437, Loss 0.0000
Epoch 438, Loss 0.0000
Epoch 439, Loss 0.0000
Epoch 440, Loss 0.0000
Epoch 441, Loss 0.0000
Epoch 442, Loss 0.0000
Epoch 443, Loss 0.0000
Epoch 444, Loss 0.0000
Epoch 445, Loss 0.0000
Epoch 446, Loss 0.0000
Epoch 447, Loss 0.0000
Epoch 448, Loss 0.0000
Epoch 449, Loss 0.0000
Epoch 450, Loss 0.0000
Epoch 451, Loss 0.0000
Epoch 452, Loss 0.0000
Epoch 453, Loss 0.0000
Epoch 454, Loss 0.0000
Epoch 455, Loss 0.0000
Epoch 

In [ ]:
# Load meta attacked data
perturbed_data = PrePtbDataset(root='/tmp/', name='cora', attack_method='meta', ptb_rate=0.10)
perturbed_adj = perturbed_data.adj
pyg_data.update_edge_index(perturbed_adj) # inplace operation

print('PERTUBATED DATA')
print(pyg_data)
print(len(pyg_data))
print(pyg_data[0])

model = Net(pyg_data.num_node_features, 512, pyg_data.num_classes).to(device)
data = pyg_data[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas = (0.9, 0.999), eps=1e-08, weight_decay = 0)

model.train()
average = []
for i in range(1, 51):
  for epoch in range(1,501):
      optimizer.zero_grad()
      out = model(data)
      loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
      print(f'Epoch {epoch:03d}, Loss {loss:.4f}')
      loss.backward()
      optimizer.step()

  model.eval()

  _, pred = model(data).max(dim=1)
  correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
  acc = correct / data.test_mask.sum().item()
  print(f'Accuracy: {acc:.4f}')
  average.append(acc)
  print(average)
print(f'Average Accuracy: {statistics.mean(average):.4f}')

In [ ]:
# Load meta attacked data
perturbed_data = PrePtbDataset(root='/tmp/', name='cora', attack_method='meta', ptb_rate=0.15)
perturbed_adj = perturbed_data.adj
pyg_data.update_edge_index(perturbed_adj) # inplace operation

print('PERTUBATED DATA')
print(pyg_data)
print(len(pyg_data))
print(pyg_data[0])

model = Net(pyg_data.num_node_features, 512, pyg_data.num_classes).to(device)
data = pyg_data[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas = (0.9, 0.999), eps=1e-08, weight_decay = 0)

model.train()
average = []
for i in range(1, 51):
  for epoch in range(1,501):
      optimizer.zero_grad()
      out = model(data)
      loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
      print(f'Epoch {epoch:03d}, Loss {loss:.4f}')
      loss.backward()
      optimizer.step()

  model.eval()

  _, pred = model(data).max(dim=1)
  correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
  acc = correct / data.test_mask.sum().item()
  print(f'Accuracy: {acc:.4f}')
  average.append(acc)
  print(average)
print(f'Average Accuracy: {statistics.mean(average):.4f}')

In [ ]:
# Load meta attacked data
perturbed_data = PrePtbDataset(root='/tmp/', name='cora', attack_method='meta', ptb_rate=0.20)
perturbed_adj = perturbed_data.adj
pyg_data.update_edge_index(perturbed_adj) # inplace operation

print('PERTUBATED DATA')
print(pyg_data)
print(len(pyg_data))
print(pyg_data[0])

model = Net(pyg_data.num_node_features, 512, pyg_data.num_classes).to(device)
data = pyg_data[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas = (0.9, 0.999), eps=1e-08, weight_decay = 0)

model.train()
average = []
for i in range(1, 51):
  for epoch in range(1,501):
      optimizer.zero_grad()
      out = model(data)
      loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
      print(f'Epoch {epoch:03d}, Loss {loss:.4f}')
      loss.backward()
      optimizer.step()

  model.eval()

  _, pred = model(data).max(dim=1)
  correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
  acc = correct / data.test_mask.sum().item()
  print(f'Accuracy: {acc:.4f}')
  average.append(acc)
  print(average)
print(f'Average Accuracy: {statistics.mean(average):.4f}')

In [ ]:
# Load meta attacked data
perturbed_data = PrePtbDataset(root='/tmp/', name='cora', attack_method='meta', ptb_rate=0.25)
perturbed_adj = perturbed_data.adj
pyg_data.update_edge_index(perturbed_adj) # inplace operation

print('PERTUBATED DATA')
print(pyg_data)
print(len(pyg_data))
print(pyg_data[0])

model = Net(pyg_data.num_node_features, 512, pyg_data.num_classes).to(device)
data = pyg_data[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas = (0.9, 0.999), eps=1e-08, weight_decay = 0)

model.train()
average = []
for i in range(1, 51):
  for epoch in range(1,501):
      optimizer.zero_grad()
      out = model(data)
      loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
      print(f'Epoch {epoch:03d}, Loss {loss:.4f}')
      loss.backward()
      optimizer.step()

  model.eval()

  _, pred = model(data).max(dim=1)
  correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
  acc = correct / data.test_mask.sum().item()
  print(f'Accuracy: {acc:.4f}')
  average.append(acc)
  print(average)
print(f'Average Accuracy: {statistics.mean(average):.4f}')